In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
sys.path

['/Users/syeehyn/Desktop/fall2020/ieor4571/personalization/eggs',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python38.zip',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/lib-dynload',
 '',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/IPython/extensions',
 '/Users/syeehyn/.ipython',
 '../']

In [4]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from tqdm.notebook import tqdm

In [5]:
from src import indexTransformer
from scipy import sparse
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [6]:
spark = Spark()

Spark UI address http://127.0.0.1:4041


In [7]:
datas = loading(spark, '../data/interim')

In [8]:
datas

{'train_0.5_0.5': DataFrame[userId: string, movieId: string, rating: string],
 'train_0.25_0.75': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.5_0.5': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.25_0.75': DataFrame[userId: string, movieId: string, rating: string],
 'train_0.75_0.25': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.75_0.25': DataFrame[userId: string, movieId: string, rating: string]}

In [9]:

training, test = cast_int(datas['train_0.75_0.25']), cast_int(datas['test_0.75_0.25'])

In [10]:
print(f'''
        training set num of rows {training.count()},
        test set num of rows {test.count()},
        training set num of users {training.select('userId').distinct().count()},
        training set num of movies {training.select('movieId').distinct().count()},
        test set num of users {test.select('userId').distinct().count()},
        test set num of movies {test.select('movieId').distinct().count()},
        ''')


        training set num of rows 89977,
        test set num of rows 30342,
        training set num of users 23503,
        training set num of movies 1102,
        test set num of users 16470,
        test set num of movies 1102,
        


In [14]:
ALS(userCol="userId", 
                itemCol="movieId", 
                ratingCol="rating").userCol

Param(parent='ALS_083d9a6c97d6', name='userCol', doc='column name for user ids. Ids must be within the integer value range.')

In [15]:
class als():
    def __init__(self, userCol, itemCol, ratingCol, regParam, seed, rank):
        self.userCol = userCol
        self.itemCol = itemCol
        self.ratingCol = ratingCol
        self.als = ALS(userCol=userCol, 
                itemCol=itemCol, 
                ratingCol=ratingCol,
                coldStartStrategy="drop", 
                nonnegative=True,
                regParam=regParam,
                seed=seed,
                rank=rank)
    def fit(self, _X):
        X = self._preprocess(_X)
        self.model = self.als.fit(X)
    
    def predict(self, _X):
        X = self._preprocess(_X)
        return self.model.transform(X)
    
    def _preprocess(self, _X):
        cast_int = lambda df: df.select([F.col(c).cast('int') for c in [self.userCol, self.itemCol]] + \
                                [F.col(self.ratingCol).cast('float')])
        return cast_int(_X)

In [16]:
als = als('userId', 'movieId', 'rating', .1, 0, 20)

In [17]:
als.fit(training)

In [20]:
als.predict(training).show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 67782|     70|   1.0| 1.0252424|
| 67919|     70|   3.0| 2.9810653|
| 68516|     70|   2.0| 1.9541498|
|124453|     70|   4.0| 3.6075344|
|163552|     70|   3.0| 2.7944746|
|195408|     70|   4.5| 3.9437785|
|201203|     70|   3.5| 3.2584264|
|208365|     70|   3.0| 2.7693505|
|252652|     70|   3.0| 2.8266904|
| 28231|     70|   5.0| 4.6779532|
| 99611|     70|   3.0| 2.9031727|
|180165|     70|   3.0| 2.8295388|
|266925|     70|   3.0| 2.9129915|
|139514|     70|   3.0|  2.971377|
|190139|     70|   4.0| 3.8798056|
|253391|     70|   1.0| 1.2994075|
|261219|     70|   3.0| 3.1057842|
|267586|     70|   4.0| 3.8791454|
|281820|     70|   4.5|  4.216389|
| 79848|     70|   2.0| 2.1279404|
+------+-------+------+----------+
only showing top 20 rows

